In [79]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt
import itertools

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: TkAgg


In [80]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [81]:
def FindTrialEndTimes(KeysSelected, timeTyping):
    # function to find start and end of tasks in experiments
    timeStartEnd = list() # format of this list will be: [startTime1, endTime1/startTime2, endTime2/startTime3, ..., endTimeN]
    
    timeStartEnd.append(timeTyping['startTime'])
    
    nTrial = 1
    
    for keys in KeysSelected:
        
        if keys[1] == 'NextPhrase':
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            timeStartEnd.append(endTimeTrial)
    
    timeStartEnd.append(timeTyping['endTime'])
    
    
    return timeStartEnd

In [82]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [83]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [84]:
def CreateTimeEpochsOfTrials(TimeStartEndMixed, UserKeys):
    # function to use list of mixed start and end times of trials and keys looked at by user to create trial epochs
    
    TimeEpochTrial = dict()
    TimeEpochTrial['Start'] = list()
    TimeEpochTrial['End'] = list()
    
    # Create list of times in userKeys to be able to use function 'nearestTimePoint'
    UserKeysStrTimes = [item3[0] for item3 in UserKeys]
    UserKeysTimes = timeConversion(UserKeysStrTimes)
    
    Flag_FoundSleepKey = 0 # Flag to indicate finding sleep key
    
    n = -1
    for time in TimeStartEndMixed:
        n = n + 1
        Flag_FoundSleepKey = 0
        
        if n == 0: # first time is only start time for the first trial
            TimeEpochTrial['Start'].append(time)
            continue
        elif n == len(TimeStartEndMixed)-1: # last time is only the end time for last trial
            
            TimeEpochTrial['End'].append(time)
            
        else: # the middle elements need to be divided into start and end
            TimeEpochTrial['End'].append(time)
            
            timeCheck = time
            
            # find the time in userkeys. Keep going to the previous element till you reach start of selection of
            # nextPhrase key
            while Flag_FoundSleepKey < 1:
                
                nearestToTrialStartTime, nearestToTrialStartInd = nearestTimePoint(UserKeysTimes, timeCheck)
                indCheck = nearestToTrialStartInd
                
                if 'NextPhrase' not in UserKeys[indCheck][1]:
                    TimeEpochTrial['Start'].append(nearestToTrialStartTime)
                    Flag_FoundSleepKey = 1
                    break
                else:
                    indCheck = indCheck - 2 # 2 added instead of 1, to allow nearestTimePoint to find the one before this
                    timeCheck = UserKeysTimes[indCheck]
                    
                
    return TimeEpochTrial      
            

In [85]:
def Convert2ColumnSizesTo1(GazeLog, pupilL_beforeDecimal, pupilL_afterDecimal, pupilR_beforeDecimal, pupilR_afterDecimal):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    PupilLogL = list()
    PupilLogR = list()
    
    PupilLogL_beforeDecimal = [item4[pupilL_beforeDecimal] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogL_afterDecimal = [item4[pupilL_afterDecimal] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_beforeDecimal = [item4[pupilR_beforeDecimal] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_afterDecimal = [item4[pupilR_afterDecimal] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
#     for i in range(0, len(PupilLogL_beforeDecimal)):
#         print('left: ', PupilLogL_beforeDecimal[i], PupilLogL_afterDecimal[i],', right: ', PupilLogR_beforeDecimal[i], 
#               PupilLogR_afterDecimal[i])
    
    for i in range(0, len(PupilLogL_beforeDecimal)):
        if 'Valid' not in PupilLogL_beforeDecimal[i] and 'Valid' not in PupilLogL_afterDecimal[i]:
            if 'nan' not in PupilLogL_beforeDecimal[i] and 'nan' not in PupilLogL_afterDecimal[i]:
                PupilLogL.append(float(PupilLogL_beforeDecimal[i]+'.'+PupilLogL_afterDecimal[i]))
            else:
                PupilLogL.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
    
    for i in range(0, len(PupilLogR_beforeDecimal)):
        if 'Valid' not in PupilLogR_beforeDecimal[i] and 'Valid' not in PupilLogR_afterDecimal[i]:
            if 'nan' not in PupilLogR_beforeDecimal[i] and 'nan' not in PupilLogR_afterDecimal[i]:
                PupilLogR.append(float(PupilLogR_beforeDecimal[i]+'.'+PupilLogR_afterDecimal[i]))
            else:
                PupilLogR.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    # if one of the pupils are nan, the other one is converted too
    nPupil = -1
    for pupilL in PupilLogL:
        nPupil = nPupil + 1
        if np.isnan(pupilL):
            if nPupil < len(PupilLogR):
                if not np.isnan(PupilLogR[nPupil]):
                    PupilLogR[nPupil] = np.nan
            else:
                del PupilLogL[len(PupilLogR):]
                
    nPupil = -1
    for pupilR in PupilLogR:
        nPupil = nPupil + 1
        if np.isnan(pupilR):
            if nPupil < len(PupilLogL):
                if not np.isnan(PupilLogL[nPupil]):
                    PupilLogL[nPupil] = np.nan
            else:
                del PupilLogR[len(PupilLogL):]
                
    #print(len(PupilLogL), len(PupilLogR))
    
    return PupilLogL, PupilLogR

In [86]:
def PupilSizeFromTrialTimes(TimeTrial, TimeGazeLog, TimeInternalGazeLog, PupilSizeLogL, PupilSizeLogR):
    # find pupil sizes from the start and end time given
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(TimeGazeLog, TimeTrial[0])
    timeEnd, timeEndInd = nearestTimePoint(TimeGazeLog, TimeTrial[1])
    
    pupilSize_TrialL = PupilSizeLogL[timeStartInd: timeEndInd]
    pupilSize_TrialR = PupilSizeLogR[timeStartInd: timeEndInd]
    
    TimeInternal_Trial = TimeInternalGazeLog[timeStartInd: timeEndInd]
    
    TimeGaze_Trial = TimeGazeLog[timeStartInd: timeEndInd]
    
    return pupilSize_TrialL, pupilSize_TrialR, TimeGaze_Trial, TimeInternal_Trial
    

In [87]:
def plotPupilSizeToCheckFiltering(pupilData, timeData, subjName):
        
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    
    ax.plot(timeData[0:min(len(timeData), len(pupilData['BeforeFilter']))], 
            pupilData['BeforeFilter'][0:min(len(timeData), len(pupilData['BeforeFilter']))], 'c')
    ax.plot(timeData[0:min(len(timeData), len(pupilData['VelocityProfile']))], 
            pupilData['VelocityProfile'][0:min(len(timeData), len(pupilData['VelocityProfile']))], 'c')
#     ax.plot(timeData[0:min(len(timeData), len(pupilData['AfterFilter']))], 
#             pupilData['AfterFilter'][0:min(len(timeData), len(pupilData['AfterFilter']))], 'b')
    #ax.plot(timeData[0:min(len(timeData), len(pupilData['AfterHampel']))], 
    #        pupilData['AfterHampel'][0:min(len(timeData), len(pupilData['AfterHampel']))], 'r')
    
    ax.set_ylabel('Absolute pupil size [in mm]')

    ax.set_title(subjName)


In [92]:
def filterBlinks(pupilData, timeListComplete, timeInS_Trial):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    # recording extra blink information - duration and frequency
    
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (23 samples) before and after the blink will also be removed
    extraBlinkSamples = 23    
    
    # checking pupil size and velocity profile of pupil size
    velocityProfile = [t - s for s, t in zip(pupilData, pupilData[1:])]
    
    pupilPlot = dict()
    pupilPlot['VelocityProfile'] = velocityProfile
    pupilPlot['BeforeFilter'] = pupilData
    
    plotPupilSizeToCheckFiltering(pupilPlot, timeInS_Trial, 'ph')
    
    # remove single missing data, that are due to hardware error
    missingVal_Single = np.argwhere(np.isnan(pupilData))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list
    missingVal_SingleDifference = [t - s for s, t in zip(missingVal_Single, missingVal_Single[1:])] # find difference 
    # between consecutive elements
    missingVal_SingleDifference.insert(0, missingVal_Single[0]) # insert the first blink index in the beginning of list
    
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index to 
    # the next nan value
    
    # first remove the single nan values, which are missing data
    eyeTracker_missingData = list() # list with index of single missing data  
    valInd = -1

    for val in missingVal_SingleDifference:
        valInd = valInd + 1
        if valInd == 0:
            continue
        if val != 1:
            if missingVal_SingleDifference[valInd-1] !=1: # if there are 2 consecutive missing values (denoted by 2 consecutive
                # non 1 numbers, they are added to the list of eyeTracker_missingData)
                eyeTracker_missingData.append(sum(missingVal_SingleDifference[:valInd]))
                
    # remove single missing values from pupil data
    pupilData_woSingleMissingData0 = [pupilData[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(pupilData))]
    pupilData_woSingleMissingData = [x for x in pupilData_woSingleMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woSingleMissingData0 = [timeListComplete[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(timeListComplete))]
    timeList_woSingleMissingData = [x for x in timeList_woSingleMissingData0 if x]
    
#     print(len(timeList_woSingleMissingData))
    
    
    
    # find the nan values again from pupilData_woSingleMissingData
    missingVal_Rest = np.argwhere(np.isnan(pupilData_woSingleMissingData))
    missingVal_Rest = list(itertools.chain.from_iterable(missingVal_Rest))
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    #print(missingVal_RestDifference)
    
    # compile and create list of start and end of blinks
    blink_missingData = dict()
    blink_missingData['Start'] = list()
    blink_missingData['End'] = list()
    
    valInd = -1
    for val in missingVal_RestDifference:
        valInd = valInd + 1
        if val > 1:
            
            #print('value', val)
            # instead of appending the actual index of blink start, since 250ms before and after the blink need to be
            # removed, it is also appended here.
            
            # just make sure that the additional samples do not make the index of blink go in negative
            if sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples > 0:
                
                blink_missingData['Start'].append(sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples)
            else:
                blink_missingData['Start'].append(0)
            
            if valInd == 0:
                continue
                
            # make sure that the additional samples do not increase the index to more than the length of the pupilData
            if sum(missingVal_RestDifference[:valInd])+extraBlinkSamples < len(pupilData_woSingleMissingData):
                blink_missingData['End'].append(sum(missingVal_RestDifference[:valInd])+extraBlinkSamples)
            else:
                blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
#         else:
#             # val is 1
#             if valInd-2 > 0 and valInd+3 < len(missingVal_RestDifference):
#                 if missingVal_RestDifference[valInd-1] > 1:
#                     if missingVal_RestDifference[valInd+1] == 1:
#                         if missingVal_RestDifference[valInd+2] > 1:
#                             print(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])], 
#                                 missingVal_RestDifference[valInd-2:valInd+3])
#                             if missingVal_RestDifference[valInd+2] > missingVal_RestDifference[valInd-1]:
#                                 if valInd-6>0:
#                                     print(missingVal_RestDifference[valInd-6:valInd+3])
#                     elif missingVal_RestDifference[valInd+1] > 1:
#                         print(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])], 
#                               missingVal_RestDifference[valInd-2:valInd+3])
#                         if missingVal_RestDifference[valInd+2] > missingVal_RestDifference[valInd-1]:
#                                 if valInd-6>0:
#                                     print(missingVal_RestDifference[valInd-6:valInd+3])
                        
                        
    # add the last blink index
    # make sure that the additional samples do not increase the index to more than the length of the pupilData
    if sum(missingVal_RestDifference)+extraBlinkSamples < len(pupilData_woSingleMissingData):
        blink_missingData['End'].append(sum(missingVal_RestDifference)+extraBlinkSamples)
    else:
        blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
                
    # need to create a list containing indexes that are to be removed
    blinkIndexList = list()
    
#     print(len(blink_missingData['Start']), len(blink_missingData['End']))
    
    # remove blinks and additional data from pupil data to get filtered data
    for indInd in range(0, len(blink_missingData['Start'])):
        blinkIndexList.append(range(blink_missingData['Start'][indInd], blink_missingData['End'][indInd]+1))
    # flatten the list
    blinkIndexList = list(itertools.chain.from_iterable(blinkIndexList))
    
    pupilData_woRestMissingData0 = [pupilData_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(pupilData_woSingleMissingData))]
    pupilData_filter = [x for x in pupilData_woRestMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woRestMissingData0 = [timeList_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(timeList_woSingleMissingData))]
    time_filter = [x for x in timeList_woRestMissingData0 if x]
    
        
        
    if np.nan in pupilData_filter:
        print('nan values in filtered data')
#         for i in enumerate(pupilData_filter):
#             print(i)
        
    
    return pupilData_filter, time_filter, blink_missingData

In [93]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [94]:
def FindAndPlotPupilSizeForEpoch(GazeLog, TimeEpochTrial):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnSizesTo1(GazeLog, -5, -4, -2, -1)
    
    # extract pupil position in decimals from the strange 2 columns for every pupil
    pupilLx, pupilRx = Convert2ColumnSizesTo1(GazeLog, 3, 4, 16, 17)
    pupilLy, pupilRy = Convert2ColumnSizesTo1(GazeLog, 5, 6, 18, 19)
    pupilLz, pupilRz = Convert2ColumnSizesTo1(GazeLog, 7, 8, 20, 21)
    
    pupilL_position = [float(np.sqrt(pupilLx[i]**2+pupilLy[i]**2+pupilLz[i]**2)) if not np.isnan(pupilLx[i]) else np.nan 
                       for i in range(0, len(pupilLx))]
    pupilL_velocity = [t - s for s, t in zip(pupilL_position, pupilL_position[1:])]
    pupilR_position = [np.sqrt(pupilRx[i]*2+pupilRy[i]**2+pupilRz[i]**2) if not np.isnan(pupilRx[i]) else np.nan 
                       for i in range(0, len(pupilRx))]
    pupilR_velocity = [t - s for s, t in zip(pupilR_position, pupilR_position[1:])]
    
    
    
#     # just to plot on same plot, pupil size and velocity profile, from Mathot 2013 -- since we have missing values in pupil position,
#     # due to no head tracking, pupil filtering cannot be performed using 
#     pupilBlinkCheck = dict()
#     pupilBlinkCheck['Left'] = pupilLogL
#     pupilBlinkCheck['Right'] = pupilL_velocity
    
#     # find difference in consecutive elements of internal time
#     timeInternalDifferenceCompl = [t - s for s, t in zip(timeInternalGazeLog, timeInternalGazeLog[1:])]
#     # divide by 1000 to make it s
#     timeOfGazeCompl = [sum(timeInternalDifferenceCompl[:i])/1000000 for i in range(1,len(timeInternalDifferenceCompl))]
    
#     plotPupilSize(pupilBlinkCheck, timeOfGazeCompl, 1)
    
    
    
    ipaList = list()
    
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(timeEpochTrial['Start'])):
                    
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['Start'][trialNr], TimeEpochTrial['End'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        # some trials were skipped, because the sentence was written before. If the time of trial is less than
        # 10s, the trial is skipped
        if timeOfGaze_Trial[-1] < 20:
            print('trial number ', trialNr+1, 'with', timeOfGaze_Trial[-1], 's will be skipped')
            continue
        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        #if trialNr == 4:
        #    for i in range(0, len(pupilSizeL_Trial)):
        #        print(pupilSizeL_Trial[i], pupilSizeR_Trial[i])
            
        #print('Trial', len(pupilSizeL_Trial), len(pupilSizeR_Trial))
        
        # smoothing of signals
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeL_Trial, timeGaze_Trial, timeOfGaze_Trial)
        pupilSizeR_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeR_Trial, timeGaze_Trial, timeOfGaze_Trial)
        

        
        #print(index_blinkEndL)
        #print(index_blinkEndR)
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        #print('After blink', len(pupilSizeL_woBlink), len(pupilSizeR_woBlink))
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3)

        pupilSize_Filter['Left'] = pupilSizeL_filter.values.tolist()
        pupilSize_Filter['Right'] = pupilSizeR_filter.values.tolist()
        
        pupilSizeL_filterList = [i[0] for i in pupilSizeL_filter.values]
        pupilSizeR_filterList = [i[0] for i in pupilSizeR_filter.values]
        
        pupilL_filterPlot = dict()
        pupilL_filterPlot['BeforeFilter'] = pupilSizeL_Trial
        pupilL_filterPlot['AfterFilter'] = pupilSizeL_woBlink
        pupilL_filterPlot['AfterHampel'] = pupilSizeL_filterList
        
        #plotPupilSizeToCheckFiltering(pupilL_filterPlot, timeOfGaze_Trial, 'ph')
        
        #print('filter', len(pupilSizeL_filterList), len(pupilSizeR_filterList))
        RLCorrelation = np.corrcoef(pupilSizeL_filterList, pupilSizeR_filterList)
        #print(RLCorrelation)
        
        
        
        

In [95]:
subjName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []

for root, dirs, subfolder in os.walk(subjName):
    if not dirs and 'ph\Test_wChinRest' in root:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        keysSelected = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or keysSelected is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                if subjName == 'sa\Test_woChinRest\p2':
                    userKeys = userKeys[:-1]
                    
                # find start time of typing
                timeTyping = OptiKeyTypingTime(userKeys)
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test1\p1' or subjName == 'rh\Test1\p1':
                    del keysSelected[-1]
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
                    del keysSelected[-1]
                    
                # divide complete data into epochs of phrases
                timeStartEndMixed = FindTrialEndTimes(keysSelected, timeTyping)
                
                # create trial time epoch using the list of start/end times of trial and userKeys, to make sure that 
                # Sleep is completely there in every trial, to allow for baseline
                timeEpochTrial = CreateTimeEpochsOfTrials(timeStartEndMixed, userKeys)
                #print(timeEpochTrial)
                
                # find and plot pupil size for every trial
                FindAndPlotPupilSizeForEpoch(gazeLog, timeEpochTrial)
                
                

ph\Test_wChinRest
trial number  11 with 18.00399 s will be skipped
